In [4]:
import requests
import os
import json
from google.cloud import storage
from dataverk_vault import api as vault_api

In [5]:
res = requests.get("https://data.ssb.no/api/klass/v1/versions/33")

In [6]:
vault_api.set_secrets_as_envs()

In [7]:
with open("/home/jovyan/creds.json", "w") as f:
    f.write(os.environ["STYRK_BUCKET_CREDS"])
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/creds.json"

In [12]:
client = storage.Client()

In [13]:
bucket = client.get_bucket("styrk-bucket")

In [14]:
blob = bucket.blob("styrk-koder/styrk.json")

In [20]:
blob.upload_from_string(json.dumps(res.json()))

## Convert to csv

In [46]:
import pandas as pd
df = pd.DataFrame.from_records(res.json()["classificationItems"])
df.to_csv("styrk.csv", index = None)#, sep=";")

In [47]:
blob = bucket.blob("styrk-koder/styrk.csv")

In [48]:
with open("styrk.csv", "rb") as my_file:
    blob.content_encoding = "utf-8"
    blob.upload_from_file(my_file)

In [24]:
blob = bucket.blob("styrk-koder/styrk.csv")

In [28]:
with open("/home/jovyan/nada-dags/styrk/data/test.csv", "wb") as f:
    blob.download_to_file(f)